In [1]:
import os
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler
import pandas as pd

In [ ]:
# Accessing the keys
# openai_mmaai_key = os.getenv('OPENAI_MMAAI_KEY')
# openai_nflai_key = os.getenv('OPENAI_NFLAI_KEY')

In [2]:
client = OpenAI()

# Code Interpreter Agent

In [3]:
fighter1 = 'Michael Page'
fighter2 = 'Ian Garry'

# fighter1 = 'Brian Ortega'
# fighter2 = 'Diego Lopes'

question = f'''{fighter1} versus {fighter2}'''

# message = f'''
# Generate a detailed prediction for the upcoming fight between {fighter1} and 
# {fighter2} with a detailed explanation of your reasoning behind it.
# '''
# message = f'''
# Research detailed statistics and historical performances of {fighter1} and {fighter2} using the CSV file data,
# and generate a detailed prediction for the fight outcome with a thorough explanation of your reasoning.
# '''
# message = input("Waiting for question: ")

In [4]:
assistant_id = "asst_TXOsgg0alTDR2qO205Su2EGE"
assistant = client.beta.assistants.retrieve(assistant_id)

# OR

# file1 = client.files.create(
#   file=open("./data/master.csv", "rb"),
#   purpose='assistants'
# )
# file2 = client.files.create(
#     file=open("./data/fighter_info.csv", "rb"),
#     purpose='assistants'
# )
# assistant = client.beta.assistants.create(
#     name="UFC Fight Analyst FINAL",
#     instructions="""
#     You are an expert handicapper for UFC/MMA fights at a major sportsbook in Las Vegas. 
#     You have a huge amount and all the historical data you need in master.csv and fighter_info.csv,
#     analyze it all and use it to help predict the outcomes of upcoming fights.
#     Your analysis should consider factors such as the fighters' past performance, weight class, 
#     method of victory, and other professional statistics and methods. 
#     Give your final fight prediction at the end, and you always must include in it the winner, 
#     method/time of victory, and detailed explanation why you think that outcome is likely.
#     """,
#     model="gpt-4o",
#     tools=[{"type": "code_interpreter"}],
#     tool_resources={
#         "code_interpreter": {
#             "file_ids": [file1.id, file2.id]
#         }
#     }
# )



### To add header information to the instructions ###
# master_df = pd.read_csv("./data/master.csv")
# fighter_info_df = pd.read_csv("./data/fighter_info.csv")
# master_headers = master_df.columns.tolist()
# fighter_info_headers = fighter_info_df.columns.tolist()
# assistant = client.beta.assistants.create(
#     name="UFC Fight Analyst FINAL",
#     instructions=f"""
#     You are an expert handicapper for UFC/MMA fights at a major sportsbook in Las Vegas. 
#     You have a huge amount and all the historical data you need in master.csv and fighter_info.csv,
#     analyze it all and use it to help predict the outcomes of upcoming fights.
#     Your analysis should consider factors such as the fighters' past performance, weight class, 
#     method of victory, and other professional statistics and methods. 
#     Give your final fight prediction at the end, and you always must include in it the winner, 
#     method/time of victory, and detailed explanation why you think that outcome is likely.
#     Here are the headers for the CSV files:
#     master.csv: {', '.join(master_headers)}
#     fighter_info.csv: {', '.join(fighter_info_headers)}
#     """,
#     model="gpt-4o",
#     tools=[{"type": "code_interpreter"}],
#     tool_resources={
#         "code_interpreter": {
#             "file_ids": [file1.id, file2.id]
#         }
#     }
# )

In [5]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question
)

def save_to_file(filename, text):
    with open(filename, 'a') as f:
        f.write(str(text) + '\n')

class EventHandler(AssistantEventHandler):    
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
        save_to_file('assistant_output.txt', text)
      
    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
        save_to_file('assistant_output.txt', delta.value)
      
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
        save_to_file('assistant_output.txt', tool_call.type)
  
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
                save_to_file('assistant_output.txt', delta.code_interpreter.input)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)
                        save_to_file('assistant_output.txt', output.logs)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

### OG event handler ###
# class EventHandler(AssistantEventHandler):    
#   @override
#   def on_text_created(self, text) -> None:
#     print(f"\nassistant > ", end="", flush=True)
      
#   @override
#   def on_text_delta(self, delta, snapshot):
#     print(delta.value, end="", flush=True)
      
#   def on_tool_call_created(self, tool_call):
#     print(f"\nassistant > {tool_call.type}\n", flush=True)
  
#   def on_tool_call_delta(self, delta, snapshot):
#     if delta.type == 'code_interpreter':
#       if delta.code_interpreter.input:
#         print(delta.code_interpreter.input, end="", flush=True)
#       if delta.code_interpreter.outputs:
#         print(f"\n\noutput >", flush=True)
#         for output in delta.code_interpreter.outputs:
#           if output.type == "logs":
#             print(f"\n{output.logs}", flush=True)


assistant > Let's analyze the upcoming fight between Michael Page and Ian Garry using the data from the provided files. I'll start by examining the contents of the files to understand the structure and ensure we have the necessary information.

Let's first load the files and inspect their contents.
assistant > code_interpreter

import pandas as pd

# Load the files
master_data = pd.read_csv('/mnt/data/file-gbwUwbwX3KbOMB0GqDaGsrp0')
fighter_info = pd.read_csv('/mnt/data/file-lUM7jhgsjX3BfoleZwoyecBm')

# Display the first few rows of each data set to understand their structure
master_data.head(), fighter_info.head()
assistant > The `master_data` contains details about various fights including event, outcome, weight class, method of victory, round, time, fighters involved, location, and date. The `fighter_info` file contains detailed professional information about the fighters such as their nicknames, birth date, nationality, hometown, association, weight class, height, wins, losses, a

In [6]:
question = 'Go much deeper and make a refined prediction.'
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question
)
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

Follow up question? go deeper and make a refined prediction 

assistant > To make a more refined prediction, I will delve deeper into specific aspects such as their striking accuracy, defense, grappling statistics, activity rate, and any significant fight trends. This will provide a clearer picture of their current form and capabilities.

### Additional Analysis:

#### Striking Metrics:
- **Striking Accuracy**: How often the fighters land their strikes.
- **Strikes Absorbed Per Minute (SApM)**: How often the fighters get hit.
- **Striking Defense**: Ability to avoid strikes.

#### Grappling Metrics:
- **Takedown Average**: Number of takedowns per 15 minutes.
- **Takedown Accuracy**: Success rate of takedown attempts.
- **Takedown Defense**: Ability to defend against takedowns.

I'll extract this additional detailed data and then compare both fighters.

Let's start by extracting these detailed metrics from the datasets.
assistant > code_interpreter

# Extract additional metrics for both

In [7]:
question = input("Follow up question? ")
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question
)
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

Follow up question? Now Brian Ortega vs Diego Lopes

assistant > Let's repeat the process for analyzing the upcoming fight between Brian Ortega and Diego Lopes. We'll first extract the relevant data for both fighters from the datasets and then perform a detailed comparison to make a prediction.

### Extract Fighter Information

I will now extract and compare the relevant information for Brian Ortega and Diego Lopes from the provided datasets.
assistant > code_interpreter

# Lookup the fighters' records and stats for Brian Ortega and Diego Lopes
ortega_info = fighter_info.loc[fighter_info['Fighter_lower'] == 'brian ortega']
lopes_info = fighter_info.loc[fighter_info['Fighter_lower'] == 'diego lopes']

# Check if we have found the fighters
ortega_info, lopes_info
assistant > Here is a summary of the key statistics for both fighters:

**Brian Ortega (T-City)**:
- Association: Black House
- Weight Class: Featherweight
- Height: 5'9"
- Record: 16 Wins, 3 Losses
  - Wins by Decision: 5
  - W

# Basic Math Tutor Example Code

In [ ]:
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

In [ ]:
# without class stuff ^

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

In [ ]:
# More template assistants code

# Upload a file with an "assistants" purpose
# file = client.files.create(
#   file=open("./data/master.csv", "rb"),
#   purpose='assistants'
# )

# Create an assistant using the file ID
# assistant = client.beta.assistants.create(
#   name="Math Tutor",
#   instructions="You are a personal math tutor. When asked a math question, write and run code to answer the question.",
#   model="gpt-4o",
#   tools=[{"type": "code_interpreter"}],
#   tool_resources={
#     "code_interpreter": {
#       "file_ids": [file.id]
#     }
#   }
# )

# thread = client.beta.threads.create(
#   messages=[
#     {
#       "role": "user",
#       "content": "I need to solve the equation `3x + 11 = 14`. Can you help me?",
#       "attachments": [
#         {
#           "file_id": file.id,
#           "tools": [{"type": "code_interpreter"}]
#         }
#       ]
#     }
#   ]
# )

# image_data = client.files.content("file-abc123")
# image_data_bytes = image_data.read()

# with open("./my-image.png", "wb") as file:
#     file.write(image_data_bytes)
    
# run_steps = client.beta.threads.runs.steps.list(
#   thread_id=thread.id,
#   run_id=run.id
# )